# COGS 108 - Final Project 

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

* [  ] YES - make available
* [ x ] NO - keep private

# Overview

Our project aims to investigate issues surrounding the spread of the Novel Coronavirus(COVID-19) in mainland China. Our investigation primarily focuses on how each provinces’ economic status correlates to the number of maximum daily increase of infected patients confirmed in that region. 

We mainly used the ordinary least squares function to find numerical supports in exploring the correlations between factors associated with regional economy (measured in Gross Regional Product(GRP), GRP growth rate, income, and population density) and the control of COVID-19 in that region (measured by maximum daily increase of number of patients confirmed). We also visualized these correlations using scatter plots to give a more concise summary.

We concluded that GRP, income and population density does positively correlate to the maximum daily increase of patients, while GRP growth rate doesn’t appear to influence the spread of COVID-19. We infer that these results are due to the fact that regions with more developed economies tend to have more interpersonal contact, therefore facilitating the spread of COVID-19.


# Names

- Yang Li
- Yiou Lyu
- Linfeng Hu
- Ruby Celeste Marroquin 

# Group Members IDs

- A15560579
- A15930345
- A15473121
- A16094382

# Research Question

How does the regional economic status of each province in mainland China correlate to its breakout and recovery of COVID-19?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

(Copy this information for each dataset)
- Dataset Name: 
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import json
import codecs
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn import linear_model

import os
import patsy
import scipy.stats as stats
import statsmodels.api as sm

import shapely.geometry as shp
from shapely.geometry import Point, Polygon
import descartes

import bs4
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

ImportError: No module named shapely.geometry

# Data Cleaning

When it came to cleaning our data, we decided to option for a generic approach: Represent our datasets in pandas DataFrames. After loading out datasets, we dropped *irrelevant* information -- when we say *irrelevant* we mean anything that doesn't pertain to what we are focusing on --  as well as any outliers in the data that may skew our results. We will also be renaming the columns to make it simpler when referencing/coding in our analyses. 

## Economic Status Data

This is the income per capita value for each province in mainland China. Income is measured in yuan. 

In [ ]:
Income = pd.read_csv('Data/Income.csv')
Income = Income.dropna(axis=1, how='all')
#Income.head()

This is the per capita Gross Regional Product value for each province. GRP per capita is measured in yuan.

In [ ]:
GRP = pd.read_csv('Data/GRP.csv')
GRP = GRP.dropna(axis=1, how='all')
#GRP.head()

## Population Density Data

Population per province here is calculated in the unit of 10000 persons. It includes all residents (permanent/temporary; rural/urban) at the end of the year that is included.

In [ ]:
population = pd.read_csv('Data/Population.csv')
population = population.dropna(axis = 1, how = 'all')
#population.head()

To calculate population density of a region, we also need the areas of each province. Here, area of each province is measured in units of square kilometers.

Since we only need the area information of each separate region, we will drop the "Toal" row at the end which contains information about the total area of China (judging by the data contained, the row name should be a typo). We will also drop the proportion row because we only need the area number. 

In [ ]:
area = pd.read_csv('Data/Area.csv')
area = area.dropna(axis = 1, how = 'all')
#shorten column names to make following analysis simpler
area = area.rename(columns={"Area (sq.km)": "Area"})
area = area[area.District != 'Toal']
area = area[area.columns[:2]]
area = area.rename(columns = {'District':'Region'})
area.head()

We also need to remove "," in the string in order to change its datatype to int64.

In [ ]:
def remove_comma(strin):
    strin = strin.replace(',','')
    return strin

area['Area']= area['Area'].apply(remove_comma)
area.head()

## Loading in Virus Data

In [ ]:
# read virus data into dataframes 
list_of_virus_data = list()

# append data between Feb 1 and Feb 25 to list
for i in range(20200201,20200226): 
    path = './Data/virus/' + str(i) + '.csv'
    df = pd.read_csv(path)
    list_of_virus_data.append(df)
    
# File 20200226.csv is missing, reason unknow. 
    
# append data between Feb 27 and Feb 29 to list
for i in range(20200227,20200230): 
    path = './Data/virus/' + str(i) + '.csv'
    df = pd.read_csv(path)
    list_of_virus_data.append(df)

# append data between Mar 1 and  Mar 1 to list
for i in range(20200301,20200302): 
    path = './Data/virus/' + str(i) + '.csv'
    list_of_virus_data.append(pd.read_csv(path))
# access ith elment in the list using list_of_virus_data[i]
# for example list_of_virus_data[0] gives the first dataframe

## Cleaning Virus Data

### Functions Used to clean Virus Data
    
The following functions were used to clean the csv files that were read in and contained various unecessary columns, characters, and such. 

In [ ]:
# remove spaces from string
def remove_space(string):
    try:
        return string.replace(" ","")
    except:
        return np.nan
    
# Find the maximum numerical number in a row
def find_max_in_a_row(df,row_index):
    num_cols = df.shape[1]
    max = 0
    for i in range(0,num_cols):
        value_in_ith_col = df.iloc[row_index,i]
        try:
            value_in_ith_col = remove_space(value_in_ith_col)
            if (int(value_in_ith_col) > max):
                max = int(value_in_ith_col)
        except:
            pass
    return max

# Find the maximum numerical numbers in each row of the dataframe, save them into df['max']
def find_max_in_a_dataframe(df):
    df['max'] = np.nan
    for i in range(0,df.shape[0]):
        df.loc[i,'max'] = find_max_in_a_row(df,i)
        
# Find the string in the data frame
# Return row_index,col_index of the string if the string is found
def find_string(df,string_to_find):
    num_rows = df.shape[0]
    num_cols = df.shape[1]
    for row_index in range(0,num_rows):
        for col_index in range(0,num_cols):
            if (df.iloc[row_index,col_index] == string_to_find):
                return row_index,col_index
    raise Exception("string_to_find not found") 
    return


# Find "Hubei" in df
# Return row_index,col_index of "Hubei" if "Hubei" is found
def find_Hubei(df):
    return find_string(df,"Hubei")

def drop_population_column(df):
    
# drop column that contains '(10,000s)'
    try:
        target_string_3 = '(10,000s)'
        target_string_3_col_index = find_string(df,target_string_3)[1]
        df = df.drop(df.columns[target_string_3_col_index],axis=1)
    except:
        pass
# drop column that contains "Population (in 10,000s)"    
    try:
        target_string_1 = "Population (in 10,000s)"
        target_string_1_col_index = find_string(df,target_string_1)[1]
        df = df.drop(df.columns[target_string_1_col_index],axis=1)
    except:
        pass
# drop column that contains "Population"
    try:
        target_string_2 = "Population"
        target_string_2_col_index = find_string(df,target_string_2)[1]
        df =df.drop(df.columns[target_string_2_col_index],axis=1)
    except:
        return df
    
# index of "Province/Region/City" = col_index of Hubei
# "Confirmed Cases" = max value in every row


def clean_df_first_pass(df):
    # find column index of "Province/Region/City"
    city_col_index = find_Hubei(df)[1]
    df["Province/Region/City"] = df.iloc[:,city_col_index]
    find_max_in_a_dataframe(df)
    # Drop all rows above Hubei
    for i in range(0, find_Hubei(df)[0]):
        df = df.drop(i)
    # Find out df["Confirmed Cases"]
    df["Confirmed Cases"] = df['max']
    df = df[["Province/Region/City","Confirmed Cases"]]
    return df



def drop_rows_whose_city_length_too_long(df,max_length):
    num_rows = df.shape[0]
    for i in range(0,num_rows):
        try:
            city = df.loc[i,'Province/Region/City']
            city_length = len(city)
            if city_length > max_length:
                df = df.drop(i)
        except:
            if not(np.isnan(city)):
                raise Exception("fail to drop the " + str(i) + "th row, which is too long")
    return df

def get_city_cases(df,region_name):
    num_rows = df.shape[0]
    return (df[df["Province/Region/City"] == region_name]["Confirmed Cases"]).iloc[0]



def merge_one_day(day):
    day_name = "Day " + str(day)
    df[day_name] = np.nan
    for i in range(0,df.shape[0]):
        #print(i)
        try:
            region_name = df.loc[i,"Province/Region/City"]
            num_cases = get_city_cases(list_of_virus_data[day],region_name)
            df.loc[i,day_name] = num_cases
        except:
            df.loc[i,day_name] = np.nan

### Grabbing Population Information

In [ ]:
list_of_virus_data[-1].head()

In [ ]:
# Get population of each province/area from WHO Report
population_from_WHO_report = list_of_virus_data[-1]
population_from_WHO_report = population_from_WHO_report[['F1',"Population"]]

# Clean population data
population_from_WHO_report.columns = ["Province/Region/City","Population" ]
population_from_WHO_report = population_from_WHO_report.drop(0)
population_from_WHO_report = population_from_WHO_report.drop(1)
population_from_WHO_report = population_from_WHO_report.drop(2)
population_from_WHO_report = population_from_WHO_report.drop(3)
population_from_WHO_report =population_from_WHO_report.reset_index(drop=True)

# Multiply populatino by 10000 for each province/region, since the unit is 10000
population_from_WHO_report = population_from_WHO_report.astype({'Population': 'int'})
population_from_WHO_report["Population"] = population_from_WHO_report["Population"] * 10000

Dropping Population Information from the dataframe about virus

In [ ]:
for i in range (0,len(list_of_virus_data)):
    try:
        list_of_virus_data[i] = drop_population_column(list_of_virus_data[i])
    except:
        pass

Cleaning the First Few Files of virus data

In [ ]:
for i in range(0,len(list_of_virus_data)):
    try:
        list_of_virus_data[i] = clean_df_first_pass(list_of_virus_data[i])
    except:
        print(i)
# get all the province names
df = list_of_virus_data[0]
for i in range (1,len(list_of_virus_data)):
    df = df.merge(list_of_virus_data[i],on="Province/Region/City",how='outer')
df = df[["Province/Region/City"]]

Drop rows whose city name is too long or too short (because these rows are not data for cities, regions, or provinces)

In [ ]:
# Drop rows whose city names are too long
df = drop_rows_whose_city_length_too_long(df,25)

### Merging Virus Data
    
Merge all the dataframes in list_of_virus_data into a comprehensive one. Rows are province/area names, columns are the number of confirmed cases every day in that province/area. 

In [ ]:
# merge all confirmed cases into df
for i in range(0,len(list_of_virus_data)):
    merge_one_day(i)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Replacing 0s with NaN for later cleaning

In [ ]:
# Fill nan in df with 0
df = df.fillna(0)

Drop Province/Region/City whose name is 0

In [ ]:
for i in range(0,df.shape[0]):
    if (df.loc[i,'Province/Region/City'] == 0):
        df = df.drop(i)       

### Dropping irrelevant and confounding information
Drop "Total". It is irrelvant to our further analysis. 

Drop "Shanxi". Because there are two provinces names "shanxi" in China. Their names are distinct in Chinese, but ambiguous in English. Thus, drop "shanxi" to reduce ambiguity. 

Drop 'MacaoSar' and 'MacaoSAR' since there's no confirmed case at all

In [ ]:
# Drop "Total". It is irrelvant to our further analysis
df = df.drop(df[df['Province/Region/City'] == "Total"].index[0])
df = df.drop(df[df['Province/Region/City'] == "Totals"].index[0])

# Drop "shanxi", because there are two provinces names "shanxi" in China. 
# Their names are distinct in Chinese, but ambiguous in English. 
# Thus, drop "shanxi" to reduce ambiguity. 
df = df.drop(df[df['Province/Region/City'] == "Shanxi"].index[0])

# Drop 'MacaoSar' & 'MacaoSAR' since there's never a COVID-19 patient appear
df = df.drop(df[df['Province/Region/City'] == "MacaoSar"].index[0])
df = df.drop(df[df['Province/Region/City'] == "MacaoSAR"].index[0])

In [ ]:
# Reset df index
df = df.reset_index(drop = True)
df
df.to_csv('merged_new_data.csv')

## Cleaning GeoSpatial Data


The file being loaded in contains the longitude and latitude values of each country and its cities, regions, states, and/or provinces that have any confirmed cases; however, for our project, we will only be focusing on China as well as our 28 day period. The code following this cell removes any data that is not relevant to our project and is included in the csv file. I'd like to acknowledge that this file is taken from the public repository from John Hopkin's university and thank them for their continuous monitoring of the current situation in regards to the coronavirus. 

### Dropping Columns and Rows

Drops the countries that do not pertain to our analysis; any country that is not China. Drops the "Country" column since the remaining country is China. Drops Shaanxi and Shanxi because their names in English may be interpreted incorrectly. 

In [ ]:
#Load files with long and lat values
coord_file = pd.read_csv('Data/covid_19_clean_complete.csv')
coord_grp = pd.read_csv('Data/grp_lat_long.csv', index_col=None)
#Drops the countries that aren't china
new_data = coord_file.drop(coord_file[coord_file['Country/Region']!='Mainland China'].index)
#Drop country column since it is only mainland china 
new_data = new_data.drop(columns='Country/Region')
#Drop these because they are not relevant to our data
new_data = new_data.drop(new_data[new_data['Province/State']=='Shaanxi'].index)
new_data = new_data.drop(new_data[new_data['Province/State']=='Shanxi'].index)

### Creating Sub-DataFrames

In [ ]:
first_date = new_data.drop(new_data[new_data['Date']!='2/1/20'].index)
first_date_exclude = first_date.drop(first_date[first_date['Province/State']=='Hubei'].index)
last_date = new_data.drop(new_data[new_data['Date']!='3/1/20'].index)
last_date_exclude = last_date.drop(last_date[last_date['Province/State']=='Hubei'].index)

### Merging GRP and Long/Lat DataFrames

This merges the dataframes together and removes any remaining duplicates that may exist inside the DataFrame. Lastly, creates two new sub-dataframes one with Hubei and one excluding Hubei. 

In [ ]:
#Used later when displaying geospatial representation of GRP 
grp_coords = pd.concat([last_date,coord_grp], axis=0, ignore_index=True)
#Drop Duplicates, keep last 
grp_coords.drop_duplicates(subset ="Province/State", keep = 'last', inplace = True)
#Only really want to use GRP, lat, long, and province/state 
grp_coords = grp_coords[['Province/State', 'Lat', 'Long','GRP']]
grp_coords_exclude = grp_coords.drop(grp_coords[grp_coords['Province/State']=='Hubei'].index)

### Loading Shape File and Setting Up Geometry
Load in the shapefile of China. Adjust the geometry according to each dataframe, as well as one for each including and excluding Hubei to further show how disproportinate it is and why we have decided to remove it. Finally creating our GeoDataFrame which allows us to plot on our China shapefile. 

In [ ]:
#Load in China's shapefile 
china_province = gpd.read_file('Data/China_Province.shp')
#Sets the geometry which is used in a GeoDataFrame for plotting points 
geometry = [Point(xy) for xy in zip(first_date['Long'], first_date['Lat'])]
geometry2 = [Point(xy) for xy in zip(first_date_exclude['Long'], first_date_exclude['Lat'])]
geometry3 = [Point(xy) for xy in zip(last_date['Long'], first_date['Lat'])]
geometry4 = [Point(xy) for xy in zip(first_date_exclude['Long'], last_date_exclude['Lat'])]
geometry5 = [Point(xy) for xy in zip(grp_coords['Long'], grp_coords['Lat'])]
geometry6 = [Point(xy) for xy in zip(grp_coords_exclude['Long'], grp_coords_exclude['Lat'])]
#Creates the geoframe datasets pertaining to their geometry 
geo_df = gpd.GeoDataFrame(first_date, crs={'init': 'epsg:4326'}, geometry=geometry)
geo_df2 = gpd.GeoDataFrame(first_date_exclude, crs={'init': 'epsg:4326'}, geometry=geometry2)
geo_df3 = gpd.GeoDataFrame(last_date, crs={'init': 'epsg:4326'}, geometry=geometry3)
geo_df4 = gpd.GeoDataFrame(last_date_exclude, crs={'init': 'epsg:4326'}, geometry=geometry4)
geo_df5 = gpd.GeoDataFrame(grp_coords, crs={'init': 'epsg:4326'}, geometry=geometry5)
geo_df6 = gpd.GeoDataFrame(grp_coords_exclude, crs={'init': 'epsg:4326'}, geometry=geometry6)

# Data Analysis & Results

## Explorative Data Analysis

### Trend of number of confirmed cases in each region (Hubei included)

In [ ]:
# trend of number of confirmed cases in each region
for i in range(0,df.shape[0]):
    df.iloc[i,:][1:].plot()

### Trend of number of confirmed cases in each region (Hubei excluded)

In [ ]:
# trend of number of confirmed cases in each region (exclude Hubei)
for i in range(1,df.shape[0]):
    df.iloc[i,:][1:].plot()

## Visualising the Outbreak with Geography

### Displaying COVID-19 Exposure by Province
    
The left side of the graphs are used to represent all regions which have been affected by the coronavirus, including Hubei. As you can see the large circle that is shown is from Hubei because it contains the most cases by far surpassing any other province. In an effort to display more *precise* data, we will be excluding Hubei -- displayed on the right. From this we can see that the data is more concise and will allow for further and simpler analysis.  

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(30, 15), constrained_layout=True)
#plots the first date including hubei 
china_province.plot(ax=ax[0][0], color='black', alpha=0.82)
geo_df.plot(column='Confirmed',ax=ax[0][0],markersize=first_date['Confirmed'], 
            color='red', marker='o', edgecolor='grey', alpha=0.5)
#plots the first date exluding hubei 
china_province.plot(ax=ax[0][1], color='black', alpha=0.82)
geo_df2.plot(column='Confirmed',ax=ax[0][1],markersize=first_date_exclude['Confirmed'], 
            color='red', marker='o', edgecolor='grey', alpha=0.5)
#plots the last date including hubei 
china_province.plot(ax=ax[1][0], color='black', alpha=0.82)
geo_df3.plot(column='Confirmed',ax=ax[1][0],markersize=last_date['Confirmed'], 
            color='red', marker='o', edgecolor='grey', alpha=0.5)
#plots the last date excluding hubei 
china_province.plot(ax=ax[1][1], color='black', alpha=0.82)
geo_df4.plot(column='Confirmed',ax=ax[1][1], markersize=last_date_exclude['Confirmed'], 
            color='red', marker='o', edgecolor='grey', alpha=0.5)
#Labels the diagrams by inclusion/exclusion of Hubei 
ax[0][0].set_title("2/1/20: Coronavirus disease (COVID-19) - Including Hubei")
ax[0][1].set_title("2/1/20: Coronavirus disease (COVID-19) - Excluding Hubei")
ax[1][0].set_title("3/1/20: Coronavirus disease (COVID-19) - Including Hubei")
ax[1][1].set_title("3/1/20: Coronavirus disease (COVID-19) - Excluding Hubei")
plt.show()

In [ ]:
# Defind functions that will be used later when analysis correlation between income and maximum infection percentage

# get the virus data of one city 
def get_virus_data_of_one_city(city):
    # find row index of the city in df
    row_index = find_string(df,city)[0]
    # get data of the desired city using row index
    city_data = df.iloc[row_index,:]
    # remove city name from city_data
    city_data = city_data[1:]
    return city_data

def get_population_data_of_one_city(city):
    # find row index of the city in df
    row_index = find_string(population_from_WHO_report,city)[0]
    # get population data of the desired city using row index
    city_population = population_from_WHO_report.iloc[row_index,1]
    return city_population

def calculate_infection_percentage_of_one_city(city):
    infected_number = get_virus_data_of_one_city(city)
    population = get_population_data_of_one_city(city)
    return infected_number/population

## Investigate correlation between income and maximum infection percentage

### Calculate infection percentage and maximum infection percentage of each region


        infection_percentage = confiremed cases / population
        maximum infection percentage = max(infection_percentage)

#### Funtions Caclulating infection percentage

In [ ]:
# Caclulate infection percentage
infection_percentage = df
row_index_of_regions_without_population = list()
for row in range(0,infection_percentage.shape[0]):
    city_name = infection_percentage.iloc[row,0]
    try:
        city_population = get_population_data_of_one_city(city_name)
        for column in range(1,infection_percentage.shape[1]):
            infection_percentage.iloc[row,column] = infection_percentage.iloc[row,column]/city_population
    except:
        row_index_of_regions_without_population.append(row)
for i in range (0,len(row_index_of_regions_without_population)):
    infection_percentage = infection_percentage.drop(row_index_of_regions_without_population[i])
infection_percentage = infection_percentage.reset_index(drop=True)        


#### Finding the maximum infection percentage


In [ ]:
# Find the maximum numerical number in a row
def find_row_max(df,row_index):
    num_cols = df.shape[1]
    max = 0.0
    for i in range(1,num_cols):
        value_in_ith_col = df.iloc[row_index,i]
        if (float(value_in_ith_col) > max):
            max = float(value_in_ith_col)
    return max

# Find the maximum numerical numbers in each row of the dataframe, save them into df['max']
def find_max(df):
    df['max'] = np.nan
    for i in range(0,df.shape[0]):
        df.loc[i,'max'] = find_row_max(df,i)

find_max(infection_percentage)
max_infection_percentage = infection_percentage[['Province/Region/City','max']]
max_infection_percentage.head()

## Correlation between max infection percentage and income

### Include Hubei and try to find correlation

In [ ]:
income = Income
income = income[['Region','2018']]
income.columns = ["Province/Region/City","income"]
income.shape

In [ ]:
analysis_df = income.merge(max_infection_percentage,how="inner")
analysis_df.head()

In [ ]:
analysis_df.corr()
outcome, predictors = patsy.dmatrices('max ~ income', analysis_df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()
print(res.summary())

#### Interpreting Outputs
p_value is too large, we fail to reject the hypothesis that maximum infection percentage of a region is uncorrelated to the people's income in that region. 

### Drop Hubei and try to find corrlation again

As we've seen before, there appears to be no apparent corelation between maximun infection percentage of a region and the citizens' income. However, we must also consider the extreme outlier -- "Hubei" province. This province is where COVID-19 first broke out, and there have been special treatments and distinguished governmental interventions placed for this province. Therefore, to diminish such comfounds and come up with a more precise correlation relationship. We decide to drop the observation of Hubei province


In [ ]:
# Drop the row that contains "Hubei"
row_index_of_Hubei = find_Hubei(analysis_df)[0]
analysis_df = analysis_df.drop(row_index_of_Hubei)
analysis_df = analysis_df.reset_index(drop = True)

# Do OLS again
outcome, predictors = patsy.dmatrices('max ~ income', analysis_df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()
print(res.summary())

#### Interpret output
Let alpha = 0.05. Since p_value (0.013) is small, we can reject the hypothesis that income is not correlated with maximum infection percentage of a region. We conclude that maxmum infection percentage of an area is correlated with people's income in that area. 

In [ ]:
## Plot the model fit line

# Plot the orginal data (as before)
plt.scatter(analysis_df['income'], analysis_df['max'], alpha=0.3, label='Data');

# Generate and plot the model fit line
xs = np.arange(analysis_df['income'].min(), analysis_df['income'].max())
ys = 2.541e-10 * xs
plt.plot(xs, ys, '--k', linewidth=4, label='Model')

plt.xlabel('Maximum Infection percentage')
plt.xlabel('Income per capita (Yuan)')
plt.legend()
plt.ylim()
plt.ylim(top = 0.00003)
plt.ylim(bottom = 0)

#### Interpret output

After excluding Hubei, even though we concluded that income does correlate with maximum infection percentage in a region, we cannot establish the relatioship between them very well using the above linear model. 

#### Rename the "province/region/city" column to align with other dataframes and make it easier for later analyses

In [ ]:
df = df.rename(columns = {'Province/Region/City':'Region'})

## Effect of Regional Gross Product(GRP) on COVID-19 outbreak

Now we use the ordinary least sqaures regression model(ols model) to figure out the correlation between GRP and COVID-19 breakout rate.

In [ ]:
#single out the GRP in 2018 for each province 
GRP_2018 = GRP[GRP.columns[:2]]
#rename max_infection_percentage's region column for later merge
max_infection_percentage_1 = max_infection_percentage.rename(columns = {'Province/Region/City':'Region'})
#merge virus breakout rate data with GRP dataframe by province
GRP_virus = pd.merge(GRP_2018, max_infection_percentage_1, on = 'Region')
#rename the columns to be more meaningful and easier for further analyses
GRP_virus = GRP_virus.rename(columns = {'2018':'GRP'})
GRP_virus = GRP_virus.rename(columns = {'CAGR':'virus'})
GRP_virus.head()

Now, we use an Ordinary Least Squares model to see the correlations between virus breakout rate and GRP in 2018 for each provinces.

In [ ]:
#set up the outcome and what we use to predict using pasty
outcome1, predictors1 = patsy.dmatrices('max ~ GRP', GRP_virus)
# Now use statsmodels to intialize an OLS linear model
# This step initializes the model, and provides the data (but not actually compute the model)
mod_GRP = sm.OLS(outcome1, predictors1)
# fit the model
res_GRP = mod_GRP.fit()
# Check out the results
print(res_GRP.summary())

Here, the correlation seems vague since we have a large p-value of 0.948. 

Now, we try to find the correlation again while exclude the case of "Hubei" province just as in the above section where we interpret relationship between income and virus breakout.

In [ ]:
#drop observation in Hubei
GRP_virus = GRP_virus[GRP_virus.Region != 'Hubei']
#set up the outcome and what we use to predict using pasty
o1, p1 = patsy.dmatrices('max ~ GRP', GRP_virus)
# Now use statsmodels to intialize an OLS linear model
# This step initializes the model, and provides the data (but not actually compute the model)
mod_GRP_dropped = sm.OLS(o1, p1)
# fit the model
res_GRP_dropped = mod_GRP_dropped.fit()
# Check out the results
print(res_GRP_dropped.summary())

#### interpret results from OLS model

As we can see here, p-value here is 0.023, smaller than the general alpha value of 0.05. Therefore we can reject the hypothesis that GRP is not correlated with maximum infection percentage of a region. It's statistically significant to conclude that the maximum infection percentage of an area is correlated with GRP per capita in that area.

Now we visualize such relationship between GRP in 2018 to the breakout rate of COVID-19.

Use polyfit to fit a 1-degree linear model, predicting virus breakout rate from GRP.

In [ ]:
# Plot the orginal data (as before)
plt.scatter(GRP_virus['GRP'], GRP_virus['max'], alpha=0.3, label='Data');

# Generate and plot the model fit line
xs = np.arange(GRP_virus['GRP'].min(), GRP_virus['GRP'].max())
# use the coefficient from OLS summary above
ys = 9.272e-11 * xs
plt.plot(xs, ys, '--k', linewidth=4, label='Model')

plt.xlabel('GRP per capita \n (yuan)')
plt.ylabel('Maximum Infected percentage \n %')
plt.legend()
plt.ylim()
plt.ylim(top = 0.00003)
plt.ylim(bottom = 0)

Next, we explore the correlation between the growth of GRP over the past 3 years in each province and the virus breakout rate

We calculate the Annual growth of GRP over 3 years according to the following formula:
        
        (GRP_new - GRP_old)/GRP_old
        

In [ ]:
#add new column annual GRP growth for the past 3 years of each province applying above formula
GRP['growth'] = (GRP['2018'] - GRP['2016']) / GRP['2016']
#single out
GRP_growth = GRP[['Region','growth']]
#merge the virus breakout rate data with GRP  growth rate by province
GRP_growth_virus = pd.merge(GRP_growth, max_infection_percentage_1, on = 'Region')
GRP_growth_virus.head()

Now, we use OLS model to see if GRP growth rate over the past 3 years is correlated with virus breakout data.

In [ ]:
#set up the outcome and what we use to predict using pasty
outcome2, predictors2 = patsy.dmatrices('max ~ growth', GRP_growth_virus)
# Now use statsmodels to initialize an OLS linear model
# This step initializes the model, and provides the data (but not actually compute the model)
mod_GRP_growth = sm.OLS(outcome2, predictors2)
# fit the model
res_GRP_growth = mod_GRP_growth.fit()
# Check out the results
print(res_GRP_growth.summary())

Again, we fail to reject the null hypothesis as p-value as large as 0.470. But that doesn't mean there's actually no correlation between GRP growth rate and virus breakout. We understand that Hubei is still a potentially extremely powerful outlier here. Therefore, we try to exclude Hubei again.

In [ ]:
#drop observation in Hubei
GRP_growth_virus = GRP_growth_virus[GRP_growth_virus.Region != 'Hubei']
#set up the outcome and what we use to predict using pasty
o3, p3 = patsy.dmatrices('max ~ growth', GRP_growth_virus)
# Now use statsmodels to initialize an OLS linear model
# This step initializes the model, and provides the data (but not actually compute the model)
mod_growth_dropped = sm.OLS(o3, p3)
# fit the model
res_growth_dropped = mod_growth_dropped.fit()
# Check out the results
print(res_growth_dropped.summary())

#### interpret results from OLS model about GRP per capita growth rate and virus breakout rate
Unfortunately, the p-value of 0.560 is still rather large, which implies that the outlier of Hubei is not the factor that influenced our investigation. GRP per capita growth rate should trully be considered uncorrelated to the virus breakout rate. We shall have more discussion about possible reasons leading up to this result in the discussion section at the end.

### Visualising GRP of each provinces

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, figsize=(30,15), sharey=True)
#Displays the GRP of each affected region using the last date (includes Hubei)
china_province.plot(ax=ax, color='black', alpha=0.82)
geo_df5.plot(column='GRP',ax=ax, cmap='cool', marker='s', edgecolor='grey', 
             legend=True, markersize=450, alpha=0.6)
#Displays the GRP of each affected region using the last date (excludes Hubei)
china_province.plot(ax=ax2, color='black', alpha=0.82)
geo_df6.plot(column='GRP',ax=ax2, cmap='cool', marker='s', edgecolor='grey', 
             markersize=450, legend=True, alpha=0.6)
ax.set_title("GRP 2018 - Including Hubei")
ax2.set_title("GRP 2018 - Including Hubei")
plt.show()

## Effect of population density on COVID-19 Outbreak (as co-factor)

### Correlation between Population density and economic status

This part is to analyze the correlation between population density and economic status. We found the data for both population and area by each province. It needs to be calculated to population density first by dividing population by area.

We decide to use the population data from the most recent year available, which is 2018.
We also need to change the column name to "district" to correspond with the area dataset.

In [ ]:
#extract the population data by only year 2018
population_2018 = population[population.columns[:2]]
population_2018.head()

we will now merge the population and area dataset by the province name. We choose to "inner" merge them because if there is a province with either no population or area, we are unable to calculate the population density.
Then we could have calculate the population density by dividing population by area.
The unit for population density is number of people per square kilometer.

In [ ]:
#merge area data and population of 2018 by region
popu_density = pd.merge(population_2018,area,on = 'Region')

#calculate the population density
popu_density['Area'] = pd.to_numeric(popu_density['Area'])
popu_density['2018'] = pd.to_numeric(popu_density['2018'])
popu_density['population density'] = popu_density['2018']/popu_density['Area']
popu_density = popu_density.drop(columns = 'Area')
popu_density = popu_density.drop(columns = '2018')
popu_density.head()

we will now merge the data for population density and economic status by province. Since we use the population data in 2018 for calculating the population density, we will also use GRP in 2018 to find the correlation between population density and economic status.

In [ ]:
#merge population density data with GRP by region
co_pd_es = pd.merge(popu_density,GRP, on = 'Region')
co_pd_es = co_pd_es[co_pd_es.columns[:3]]
#rename the 2018 GRP column
co_pd_es = co_pd_es.rename(columns = {'2018':'GRP'})
co_pd_es.head()

Now, we look into the correlation between population density and GRP to see if they're actually correlated. We use the Pasty statsmodel to quantify our results.

In [ ]:
co_pd_es = co_pd_es.rename(columns = {'population density':'density'})
#we want to see how well does population density and GRP correlate using pasty
outcome, predictors = patsy.dmatrices('density ~ GRP', co_pd_es)
# Now use statsmodels to intialize an OLS linear model
#  This step initializes the model, and provides the data (but not actually compute the model)
mod = sm.OLS(outcome, predictors)
# fit the model
res = mod.fit()
# Check out the results
print(res.summary())

As we can see from the coefficient and R-squared value above, there's a positive relationship between population density and region gross product. 

Now we can visualize the correlation between population density and GRP . we will make a scatter plot of GRP vs. population density and draw the linear regression line to show the relationship.

In [ ]:
#finding the coefficient for the fit line using polyfit
a1 = np.polyfit(co_pd_es['density'],co_pd_es['GRP'],deg = 1)[0]
b1 = np.polyfit(co_pd_es['density'],co_pd_es['GRP'],deg = 1)[1]
a1,b1

In [ ]:
#plot the original data
plt.scatter(co_pd_es['density'],co_pd_es['GRP'],alpha =0.3, label = 'Data')

#Generate and plot the model fit line
pred_GRP = co_pd_es['density']*a1+b1
plt.plot(co_pd_es['density'],pred_GRP,'--k',label = 'Model')

plt.title('GRP vs. Population Density by Province')
plt.xlabel("population density \n number of people per square km" )
plt.ylabel("GRP (yuan)")
plt.legend()

From this plot, it is clear that there is a positive relationship between population density and GRP. The regions with higher population density tend to have higher GRP. Thus, population density might also serve as a factor that confounds with the economic status. 

With better GRP, the province are expected to have a slower breakout rate due to the the better infrastructures like hospitals or clinics to support the citizens against the cirus. However, the breakout rates are actually tend to be higher with province with higher GRP. Population density might be the most important cofounding factor for this controversy. 

Now, we will look at the correlation between population density and the virus breakout rate to see whether they are in fact correlated.

## correlation between population density and maximum infection percentage

In [ ]:
#merge population density data with virus breakout rate by region
PD_virus = pd.merge(popu_density, max_infection_percentage_1, on = 'Region')
#rename the population density column
PD_virus = PD_virus.rename(columns = {'population density':'PD'})
PD_virus.head()


We will use an Ordinary Least Squares model to find the correlation between population density and the virus breakout rate in 2018 for each provinces.

In [ ]:
#set up the outcome and what we use to predict using pasty
npvoutcome1, npvpredictors1 = patsy.dmatrices('max~ PD',PD_virus)
#now use statsmodels to initialize an OLS linear model
#this step initializes the model, and provides the data without actually computes the model
mod_PD = sm.OLS(npvoutcome1,npvpredictors1)
#fit the model
res_PD = mod_PD.fit()
#check out the results
print(res_PD.summary())

The correlation seems vague because we have a large p-value of 0.859.

Now we try to find the correlation again while exclude the case of "Hubei" province just as in the above section.

In [ ]:
#drop observation in Hubei
PD_virus = PD_virus[PD_virus.Region != 'Hubei']
#set up the outcome and what we use to predict using pasty
npvo1, npvp1 = patsy.dmatrices('max ~ PD', PD_virus)
# Now use statsmodels to intialize an OLS linear model
# This step initializes the model, and provides the data (but not actually compute the model)
mod_PD_dropped = sm.OLS(npvo1, npvp1)
# fit the model
res_PD_dropped = mod_PD_dropped.fit()
# Check out the results
print(res_PD_dropped.summary())

#### interpret results from OLS model

As we can see here, p-value here is 0.029, smaller than the general alpha value of 0.05. Therefore we can reject the hypothesis that population density is not correlated with maximum infection percentage of a region. It's statistically significant to conclude that the maximum infection percentage of an area is correlated with population density per capita in that area. In such case, we confirm that the population density serve as a cofactor that contributes to the correlation between GRP and maximum infection percentage.

Now we visualize such relationship between population density in 2018 to the breakout rate of COVID-19.

Use polyfit to fit a 1-degree linear model, predicting virus breakout rate from population density.


In [ ]:
# Plot the orginal data (as before)
plt.scatter(PD_virus['PD'], PD_virus['max'], alpha=0.3, label='Data');

# Generate and plot the model fit line
# use the coefficient from OLS summary above
PDmodel = PD_virus['PD']*4.635e-5 
plt.plot(PD_virus['PD'],PDmodel,'--k',label = 'Model')

plt.xlabel('population density \n number of people per square km')
plt.ylabel('Maximum Infected percentage \n %')
plt.legend()
plt.ylim()
plt.ylim(top = 0.00003)
plt.ylim(bottom = 0)

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*